In [ ]:
%matplotlib inline

import time
import json
import shutil

import h5py
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics.cluster

import torch

# filename = 'hists/hypprop_with_predictor_eeb99_hughtok1_20180921_191657.h5'
# filename = 'hists/hypprop_with_predictor_eeb100_hughtok2_20180921_193244.h5'
# filename = 'hists/hypprop_with_predictor_eeb101_hughtok3_20180921_193516.h5'
# filename = 'hists/hypprop_with_predictor_eeb102_hughtok4_20180921_193310.h5'

# filename = 'hists/hypprop_with_predictor_eeb103_hughtok1_20180921_212947.h5'

# filename = 'hists/hypprop_with_predictor_eeb105_hughtok5_20180921_230949.h5'
# filename = 'hists/hypprop_with_predictor_eeb106_hughtok6_20180921_231010.h5'

# filename = 'hists/hypprop_with_predictor_eeb107_hughtok2_20180922_013551.h5'
# filename = 'hists/hypprop_with_predictor_eeb108_hughtok1_20180922_170013.h5'
# filename = 'hists/hypprop_with_predictor_eeb109_hughtok2_20180922_184537.h5'
# filename = 'hists/hypprop_with_predictor_eeb113_hughtok6_20180922_211153.h5'

# filename = 'hists/hypprop_with_predictor_eeb115_hughtok1_20180923_173818.h5'
# filename = 'hists/hypprop_with_predictor_eeb116_hughtok2_20180923_175444.h5'
# filename = 'hists/hypprop_with_predictor_eeb117_hughtok3_20180923_175444.h5'

# filename = 'hists/hypprop_with_predictor_eeb132_hughtok3_20180925_014550.h5'

# filename = 'hists/hypprop_with_predictor_eeb135_hughtok6_20180925_082442.h5'
# filename = 'hists/hypprop_with_predictor_eec16_hughtok1_20180927_011347.h5'
filename = 'hists/hypprop_eec79_hughtok5_20181006_135449.h5'

def run(filename):
    shutil.copyfile(f'../{filename}', '/tmp/foo.h5')
    f = h5py.File('/tmp/foo.h5', 'r')

    hypotheses_train = torch.from_numpy(f['hypotheses_train'][:])
    hypotheses_gnd_train = torch.from_numpy(f['gnd_hypotheses_train'][:].astype(np.uint8)).long()
    dsrefs_train = torch.from_numpy(f['dsrefs_train'][:].astype(np.uint8)).long()
    resdicts = f['resdicts']
    
    meta = json.loads(f['meta'][0])
    ref = meta['ref']

    U_gnd = hypotheses_gnd_train.size(0)
    U_pred = hypotheses_train.size(0)

    N = dsrefs_train.size(0)

    render_start_id = 0
    render_end_id_excl = len(resdicts)

    batch_size = 128

    print('hypotheses_train.size()', hypotheses_train.size())
    _hypotheses_train = hypotheses_train[:, -32:].contiguous()
    _hypotheses_gnd_train = hypotheses_gnd_train[:, -4:].contiguous()
    
    _N = _hypotheses_train.view(-1).size(0)
    print('_N', _N)
    plt.scatter(np.arange(_N), _hypotheses_train.view(-1).numpy())
    plt.title(ref)
    plt.show()
    
    for u in range(U_pred):
        _h = _hypotheses_train[u].contiguous().view(-1)
        _N = _h.size(0)
        plt.scatter(np.arange(_N) + _N * u, _h.numpy(), s=20)
    plt.title(ref)
    plt.show()

    for u in range(U_pred):
        _h = _hypotheses_train[u].contiguous().view(-1)
        _N = _h.size(0)
        plt.scatter(np.arange(_N), _h.numpy(), s=20, label=str(u))
    plt.legend()
    plt.title(ref)
    plt.show()

    for u in range(U_pred):
        _h = _hypotheses_train[u].contiguous().view(-1)
        _N = _h.size(0)
        plt.scatter(np.arange(_N), _h.numpy(), s=20)
        plt.title(ref)
        plt.show()

#     for u_gnd in range(U_gnd):
    V = _hypotheses_train.size(-1)
    print('V', V)
    for gnd_symbol in range(V):
#         print('_hypotheses_gnd_train[0]', _hypotheses_gnd_train[0])
        _idxes = (_hypotheses_gnd_train[0] == gnd_symbol).view(-1).nonzero().view(-1).long()
#         print('_idxes', _idxes)
        if len(_idxes.size()) != 1 or _idxes.size(0) == 0:
            continue
        _h = _hypotheses_train[:, _idxes]
        print('gnd_symbol', gnd_symbol)
        for u in range(U_pred):
            _hh = _h[u].contiguous().view(-1)
            _N = _hh.size(0)
            plt.scatter(np.arange(_N) + _N * u, _hh.numpy(), s=20)
        plt.show()

#         plt.figure(figsize=(10.0 * U_gnd, 0.15 * num_renders))
#         plt.cla()
#         for u_gnd in range(U_gnd):
#             plt.subplot(1, 5, u_gnd + 1)
#             ami = amis[:, :, u_gnd]
# #             print('u_gnd', u_gnd, 'ami.size()', ami.size())
#             plt.imshow(ami.numpy())
#         plt.show()
    f.close()

run(filename)